In [12]:
import math
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from scipy.sparse import find
from IPython.display import clear_output

#nltk.download('punkt')
#nltk.download('stopwords')

# Cosine Similarity Scheme
*add description here*

In [13]:
# creating the dataframe with original skincare products dataset

products = pd.read_csv('data/skincare_products.csv')
products.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [14]:
def cosine_similarity(X, Y):

  # establish stop words
  X_list = word_tokenize(X)
  Y_list = word_tokenize(Y)
  sw = stopwords.words('english')
  l1 =[];l2 =[]

  # remove stop words from strings
  X_set = {w for w in X_list if not w in sw}
  Y_set = {w for w in Y_list if not w in sw}

  # form set with words from both strings
  rvector = X_set.union(Y_set)
  for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
  c = 0

  # cosine similarity formula
  for i in range(len(rvector)):
    c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)

  return cosine

In [15]:
products.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [16]:
# testing cosine_similarity function on products in database

# comparing first and second products in the skincare products database
test = cosine_similarity(products.iloc[0, 5], products.iloc[1, 5])
print(test)

# comparing first and third products in the skincare products database
test2 = cosine_similarity(products.iloc[0, 5], products.iloc[2, 5])
print(test2)

# comparing first and fourth products in the skincare products database
test2 = cosine_similarity(products.iloc[0, 5], products.iloc[3, 5])
print(test2)

print('Out of these comparisons, products 1 and 4 are most similar with this method')

0.22555354977384037
0.1487044791289829
0.5283120976241462
Out of these comparisons, products 1 and 4 are most similar with this method


* Create a 1472*1472 matrix of all of the similarity scores, i.e., the entry in row 1 column 2 should contain the cosine similarity between product 1 and product 2 

In [17]:
'''
cosine_similarity_matrix = np.zeros((products.shape[0], products.shape[0]))

for i in range(products.shape[0]):
    print("Generating scores for product", i)
    clear_output(wait=True)
    for j in range(i+1, products.shape[0]):
        cosine_similarity_matrix[i,j] = cosine_similarity(products.iloc[i,5], products.iloc[j,5])

cosine_similarity_matrix[0:5, 0:5]
'''

'\ncosine_similarity_matrix = np.zeros((products.shape[0], products.shape[0]))\n\nfor i in range(products.shape[0]):\n    print("Generating scores for product", i)\n    clear_output(wait=True)\n    for j in range(i+1, products.shape[0]):\n        cosine_similarity_matrix[i,j] = cosine_similarity(products.iloc[i,5], products.iloc[j,5])\n\ncosine_similarity_matrix[0:5, 0:5]\n'

In [18]:
# np.save("data/cosine_similarity_matrix.npy",cosine_similarity_matrix)

In [19]:
cosine_similarity_matrix = np.load("data/cosine_similarity_matrix.npy")

for i in range(products.shape[0]):
    for j in range(i+1, products.shape[0]):
        cosine_similarity_matrix[j,i] = cosine_similarity_matrix[i,j]

np.fill_diagonal(cosine_similarity_matrix, 1)

cosine_similarity_matrix[0:3, 0:3]

array([[1.        , 0.22555355, 0.14870448],
       [0.22555355, 1.        , 0.23268946],
       [0.14870448, 0.23268946, 1.        ]])

In [26]:
def get_5_most_similar_products(df, cos_sim_matrix: np.ndarray, index: int) -> np.ndarray:
    '''
    Gets the 5 most similar products (aside from itself) of the 
    product at the given position using the weighted distance matrix. 
    Returns a list.
    weighted_similarity_matrix: the matrix of similarity scores
    index: the positional index of the product
    '''
    sorted_indices = sorted(range(len(cos_sim_matrix[index])),
                        key = lambda i: cos_sim_matrix[index][i],
                        reverse = True)
    
    most_similar_products = df.iloc[sorted_indices[1:6],2].values
    most_similar_scores = np.round(cos_sim_matrix[index, sorted_indices[1:6]], decimals=2)

    return (most_similar_products, most_similar_scores)

In [27]:
get_5_most_similar_products(products, cosine_similarity_matrix, 1)

(array(['Facial Treatment Essence Mini',
        'Facial Treatment Essence Karan Singh Limited Edition',
        'Facial Treatment Clear Lotion Toner',
        'Brightening Derm Revival Mask', 'Facial Treatment Mask'],
       dtype=object),
 array([1.  , 0.85, 0.68, 0.62, 0.59]))

In [11]:
products['Weighted Similarity Products'] = \
    products.apply(lambda x: get_5_most_similar_products(products, cosine_similarity_matrix, x)[0])

products['Weighted Similarity Scores'] = \
    products.apply(lambda x: get_5_most_similar_products(products, cosine_similarity_matrix, x)[1])
    
products.head(3)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices